In [33]:
import numpy as np

# Load data
data = np.load("predictions.npy")  # shape = (batch_no, loc/vel, dataset_in_batch, n_nodes, timesteps, 3_directions)
coords = data[1, 0, 1, :, :, :]  # Extract coordinates
vel = data[1, 1, 1, :, :, :]  # Extract velocities
connect = np.load("connectivity.npy")  # Load connectivity data
ntimesteps = coords.shape[1]

# Define the Tecplot file writing function
def write_tecplot_dat(filename, coords, vel, connectivity):
  n_nodes = coords.shape[0]
  n_elements = connectivity.shape[0]

  with open(filename, 'w') as f:
    f.write('TITLE="3D TRIANGULAR SURFACE DATA"\n')
    f.write('VARIABLES="X","Y","Z"\n')
    f.write(f'ZONE T="unstruc" N={n_nodes} E={n_elements} F=FEPOINT ET=TRIANGLE\n')

    # Write node data
    for i in range(n_nodes):
      x, y, z = coords[i, :]
      u, v, w = vel[i, :]
      f.write(f'{x} {y} {z}\n')

    # Write connectivity data
    for conn in connectivity:
      f.write(f'{conn[0] + 1} {conn[1] + 1} {conn[2] + 1}\n')  # Tecplot uses 1-based indexing

# Write the data to a Tecplot .dat file
for t in range(ntimesteps):   
  write_tecplot_dat("./postproc/output_%i.dat"%t, coords[:,t,:], vel[:,t,:], connect)